In [1]:
!pip install torch torchvision torchaudio
!pip install ogb
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.

In [ ]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

wandb.login(key="Your-api-key")

# *BFS Positional Encodings with Variable k-Roots in GNNs*

In [5]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = "GCN Graph classification(Protein)_Gal"
ENTITY = "noynetanel1-tel-aviv-university"
K_VALUES = [2,3,5,8,10]

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, bfs_dim=8, k=3):
        super().__init__()
        self.bfs_proj = torch.nn.Sequential(
            torch.nn.Linear(k, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, bfs_dim)
        )
        self.conv1 = GCNConv(in_channels + bfs_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, bfs_ranks, edge_index, batch):
        bfs_feat = self.bfs_proj(bfs_ranks)
        x = torch.cat([x, bfs_feat], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def make_bfs_transform(k):
    def transform(data):
        G = to_networkx(data, to_undirected=True)
        bfs_ranks = torch.full((data.num_nodes, k), -1.0, dtype=torch.float)
        if G.number_of_nodes() > 0:
            degrees = dict(G.degree())
            top_k_nodes = sorted(degrees, key=degrees.get, reverse=True)[:k]
            for idx, root in enumerate(top_k_nodes):
                bfs_order = list(nx.bfs_tree(G, root))
                L = max(1, len(bfs_order))
                for i, node in enumerate(bfs_order):
                    bfs_ranks[node, idx] = i / L
        data.bfs = bfs_ranks
        if data.x is None:
            data.x = torch.ones((data.num_nodes, 1))  # default feature
        return data
    return transform

def run_one_k(k_roots):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    for SEED in range(1, NUM_SEEDS + 1):
        set_seed(SEED)

        # Load dataset safely
        try:
            original_dataset = TUDataset(
                root='data/TUDataset',
                name='PROTEINS',
                transform=make_bfs_transform(k_roots)
            )
        except Exception as e:
            print(f"Error loading dataset: {e}")
            return

        # Filter None entries but keep original_dataset for num_classes
        dataset = [d for d in original_dataset if d is not None]
        if len(dataset) == 0:
            print("No valid graphs in dataset")
            return

        # Shuffle dataset
        random.shuffle(dataset)

        # Ensure features exist
        if dataset[0].x is None:
            for data in dataset:
                data.x = torch.ones((data.num_nodes, 1))

        # Get num_classes from the original dataset
        num_classes = original_dataset.num_classes
        if num_classes is None:
            num_classes = int(max([d.y.max().item() for d in dataset]) + 1)

        num_graphs = len(dataset)
        train_size = int(0.8 * num_graphs)
        train_dataset = dataset[:train_size]
        test_dataset = dataset[train_size:]

        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

        wandb.init(
            project=PROJECT,
            entity=ENTITY,
            name=f"GCN_BFS_MaxDeg_{k_roots}Roots_seed_{SEED}",
            group="GCN_PROTEINS_BFS_K_Roots",
            job_type="bfs-k-roots",
            reinit=True,
            config={
                "model": "GCN",
                "hidden_channels": 64,
                "epochs": EPOCHS,
                "lr": 0.001,
                "dataset": "PROTEINS",
                "task": f"BFS_{k_roots}_Roots",
                "bfs_k_roots": k_roots,
                "bfs_root_criterion": "max_degree",
                "seed": SEED,
                "cuda_available": torch.cuda.is_available(),
                "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
                "cuda_version": torch.version.cuda if torch.cuda.is_available() else None
            }
        )

        model = GCN(
            in_channels=dataset[0].x.shape[1],
            hidden_channels=64,
            out_channels=num_classes,
            k=k_roots
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

        def train():
            model.train()
            total_loss = 0.0
            for data in train_loader:
                data = data.to(device)
                out = model(data.x, data.bfs, data.edge_index, data.batch)
                loss = F.nll_loss(out, data.y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                total_loss += loss.item() * data.num_graphs
            return total_loss / len(train_dataset)

        def test():
            model.eval()
            correct = 0
            for data in test_loader:
                data = data.to(device)
                out = model(data.x, data.bfs, data.edge_index, data.batch)
                pred = out.argmax(dim=1)
                correct += int((pred == data.y).sum())
            return correct / len(test_dataset)

        for epoch in range(1, EPOCHS + 1):
            loss = train()
            test_acc = test()
            # compute train accuracy
            model.eval()
            correct_train = 0
            for data in train_loader:
                data = data.to(device)
                out = model(data.x, data.bfs, data.edge_index, data.batch)
                pred = out.argmax(dim=1)
                correct_train += int((pred == data.y).sum())
            train_acc = correct_train / len(train_dataset)
            print(f"K={k_roots} | Seed {SEED} | Epoch {epoch:03d} | Loss: {loss:.4f} | Test Acc: {test_acc:.4f}")
            wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

        wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for k in K_VALUES:
        run_one_k(k)


K=2 | Seed 1 | Epoch 001 | Loss: 0.6840 | Test Acc: 0.5964
K=2 | Seed 1 | Epoch 002 | Loss: 0.6732 | Test Acc: 0.5964
K=2 | Seed 1 | Epoch 003 | Loss: 0.6665 | Test Acc: 0.6099
K=2 | Seed 1 | Epoch 004 | Loss: 0.6622 | Test Acc: 0.6278
K=2 | Seed 1 | Epoch 005 | Loss: 0.6584 | Test Acc: 0.6323
K=2 | Seed 1 | Epoch 006 | Loss: 0.6471 | Test Acc: 0.6368
K=2 | Seed 1 | Epoch 007 | Loss: 0.6422 | Test Acc: 0.6637
K=2 | Seed 1 | Epoch 008 | Loss: 0.6261 | Test Acc: 0.6816
K=2 | Seed 1 | Epoch 009 | Loss: 0.6073 | Test Acc: 0.6906
K=2 | Seed 1 | Epoch 010 | Loss: 0.5835 | Test Acc: 0.6951
K=2 | Seed 1 | Epoch 011 | Loss: 0.5672 | Test Acc: 0.7220
K=2 | Seed 1 | Epoch 012 | Loss: 0.5580 | Test Acc: 0.7175
K=2 | Seed 1 | Epoch 013 | Loss: 0.5516 | Test Acc: 0.7309
K=2 | Seed 1 | Epoch 014 | Loss: 0.5478 | Test Acc: 0.7578
K=2 | Seed 1 | Epoch 015 | Loss: 0.5390 | Test Acc: 0.6996
K=2 | Seed 1 | Epoch 016 | Loss: 0.5391 | Test Acc: 0.7444
K=2 | Seed 1 | Epoch 017 | Loss: 0.5379 | Test Acc: 0.72

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,██▇▇▇▆▆▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▂▁▁▂▁▂▁▁▁▁▁
test_acc,▁▁▂▂▃▄▅▅▅▆█▅▇▆▇▇▇▇▇▆▇▇▇▇▆▇▇▇▆▇▇▇▇▇▆▇▆▇▇▆
train_acc,▁▁▁▁▂▅▄▆▅▇▇▇▇▇▇▇█▇▇▇▇████▇██████████████
epoch,50
loss,0.51422
test_acc,0.71749
train_acc,0.76966


K=2 | Seed 2 | Epoch 001 | Loss: 0.6714 | Test Acc: 0.5695
K=2 | Seed 2 | Epoch 002 | Loss: 0.6676 | Test Acc: 0.6054
K=2 | Seed 2 | Epoch 003 | Loss: 0.6664 | Test Acc: 0.5695
K=2 | Seed 2 | Epoch 004 | Loss: 0.6632 | Test Acc: 0.5785
K=2 | Seed 2 | Epoch 005 | Loss: 0.6568 | Test Acc: 0.5740
K=2 | Seed 2 | Epoch 006 | Loss: 0.6601 | Test Acc: 0.5919
K=2 | Seed 2 | Epoch 007 | Loss: 0.6526 | Test Acc: 0.6413
K=2 | Seed 2 | Epoch 008 | Loss: 0.6408 | Test Acc: 0.5919
K=2 | Seed 2 | Epoch 009 | Loss: 0.6321 | Test Acc: 0.6637
K=2 | Seed 2 | Epoch 010 | Loss: 0.6194 | Test Acc: 0.6726
K=2 | Seed 2 | Epoch 011 | Loss: 0.6060 | Test Acc: 0.6457
K=2 | Seed 2 | Epoch 012 | Loss: 0.5944 | Test Acc: 0.7040
K=2 | Seed 2 | Epoch 013 | Loss: 0.5787 | Test Acc: 0.7309
K=2 | Seed 2 | Epoch 014 | Loss: 0.5724 | Test Acc: 0.7309
K=2 | Seed 2 | Epoch 015 | Loss: 0.5598 | Test Acc: 0.7354
K=2 | Seed 2 | Epoch 016 | Loss: 0.5637 | Test Acc: 0.6682
K=2 | Seed 2 | Epoch 017 | Loss: 0.5488 | Test Acc: 0.71

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇█▇▆▆▅▅▃▃▃▂▂▂▂▂▂▂▂▁▂▂▁▁▂▂▂▁▁▁▂▁▁▁▁▁▁▁
test_acc,▁▂▁▁▁▂▄▄▄▅▆▄▆▆█▇▆▇▆▇▆▆▇█▇▇█▅█▇▆█▆███▇▆▆█
train_acc,▁▂▁▂▂▄▂▄▅▅▇▇▅▆▅▇▇▇▇▆▇▇▇█▇██▆█▇▇█▇███▇█▇█
epoch,50
loss,0.52074
test_acc,0.78027
train_acc,0.7573


K=2 | Seed 3 | Epoch 001 | Loss: 0.6736 | Test Acc: 0.5919
K=2 | Seed 3 | Epoch 002 | Loss: 0.6665 | Test Acc: 0.5919
K=2 | Seed 3 | Epoch 003 | Loss: 0.6616 | Test Acc: 0.6099
K=2 | Seed 3 | Epoch 004 | Loss: 0.6537 | Test Acc: 0.6188
K=2 | Seed 3 | Epoch 005 | Loss: 0.6481 | Test Acc: 0.6143
K=2 | Seed 3 | Epoch 006 | Loss: 0.6380 | Test Acc: 0.6233
K=2 | Seed 3 | Epoch 007 | Loss: 0.6422 | Test Acc: 0.6233
K=2 | Seed 3 | Epoch 008 | Loss: 0.6260 | Test Acc: 0.6951
K=2 | Seed 3 | Epoch 009 | Loss: 0.6131 | Test Acc: 0.7354
K=2 | Seed 3 | Epoch 010 | Loss: 0.6016 | Test Acc: 0.6996
K=2 | Seed 3 | Epoch 011 | Loss: 0.5884 | Test Acc: 0.7085
K=2 | Seed 3 | Epoch 012 | Loss: 0.5731 | Test Acc: 0.7309
K=2 | Seed 3 | Epoch 013 | Loss: 0.5601 | Test Acc: 0.7085
K=2 | Seed 3 | Epoch 014 | Loss: 0.5700 | Test Acc: 0.6951
K=2 | Seed 3 | Epoch 015 | Loss: 0.5506 | Test Acc: 0.7130
K=2 | Seed 3 | Epoch 016 | Loss: 0.5526 | Test Acc: 0.7175
K=2 | Seed 3 | Epoch 017 | Loss: 0.5365 | Test Acc: 0.71

...

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▆▆▆▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▂▁▁▁
test_acc,▁▄▄▄▇█▄▇▃▅▅▆▄▄▅▇▅▆▃▇▅▅▄▅▄▄▅▅▄▄▃▄▄▄▄▂▃▆▂▇
train_acc,▁▃▃▃▄▅▆▅▅▆▆▆▅▆▆▆▆▆▆▇▆▇▆▇▇▆▇▇▅▇▇▇████████
epoch,50
loss,0.48661
test_acc,0.69955
train_acc,0.76854


K=10 | Seed 29 | Epoch 001 | Loss: 0.6775 | Test Acc: 0.6323
K=10 | Seed 29 | Epoch 002 | Loss: 0.6445 | Test Acc: 0.6906
K=10 | Seed 29 | Epoch 003 | Loss: 0.6184 | Test Acc: 0.7040
K=10 | Seed 29 | Epoch 004 | Loss: 0.6023 | Test Acc: 0.7040
K=10 | Seed 29 | Epoch 005 | Loss: 0.5948 | Test Acc: 0.7265
K=10 | Seed 29 | Epoch 006 | Loss: 0.5899 | Test Acc: 0.7085
K=10 | Seed 29 | Epoch 007 | Loss: 0.5860 | Test Acc: 0.7265
K=10 | Seed 29 | Epoch 008 | Loss: 0.5799 | Test Acc: 0.7309
K=10 | Seed 29 | Epoch 009 | Loss: 0.5761 | Test Acc: 0.7265
K=10 | Seed 29 | Epoch 010 | Loss: 0.5744 | Test Acc: 0.7220
K=10 | Seed 29 | Epoch 011 | Loss: 0.5671 | Test Acc: 0.7220
K=10 | Seed 29 | Epoch 012 | Loss: 0.5647 | Test Acc: 0.7265
K=10 | Seed 29 | Epoch 013 | Loss: 0.5632 | Test Acc: 0.7309
K=10 | Seed 29 | Epoch 014 | Loss: 0.5595 | Test Acc: 0.7309
K=10 | Seed 29 | Epoch 015 | Loss: 0.5637 | Test Acc: 0.7265
K=10 | Seed 29 | Epoch 016 | Loss: 0.5621 | Test Acc: 0.7399
K=10 | Seed 29 | Epoch 0

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▃▂▂▂▂▂▂▂▂▁▁▂▂▂▂▁▁▁▁
test_acc,▁▄▅▅▆▆▇▆▆▆▇▇▆▇█▆▇▆▇█▆▇▇▇▇▇▇▇█▇█▇▇▇▇▇█▇█▇
train_acc,▁▄▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▆▇▇▇▇▇▆▇▇▇▇▆█▇▇▇▇███▇█▇
epoch,50
loss,0.5161
test_acc,0.73094
train_acc,0.73483


K=10 | Seed 30 | Epoch 001 | Loss: 0.6697 | Test Acc: 0.6143
K=10 | Seed 30 | Epoch 002 | Loss: 0.6312 | Test Acc: 0.6278
K=10 | Seed 30 | Epoch 003 | Loss: 0.6055 | Test Acc: 0.6323
K=10 | Seed 30 | Epoch 004 | Loss: 0.5890 | Test Acc: 0.6278
K=10 | Seed 30 | Epoch 005 | Loss: 0.5819 | Test Acc: 0.6278
K=10 | Seed 30 | Epoch 006 | Loss: 0.5743 | Test Acc: 0.6502
K=10 | Seed 30 | Epoch 007 | Loss: 0.5711 | Test Acc: 0.6502
K=10 | Seed 30 | Epoch 008 | Loss: 0.5623 | Test Acc: 0.6457
K=10 | Seed 30 | Epoch 009 | Loss: 0.5603 | Test Acc: 0.6233
K=10 | Seed 30 | Epoch 010 | Loss: 0.5584 | Test Acc: 0.6502
K=10 | Seed 30 | Epoch 011 | Loss: 0.5494 | Test Acc: 0.6457
K=10 | Seed 30 | Epoch 012 | Loss: 0.5469 | Test Acc: 0.6547
K=10 | Seed 30 | Epoch 013 | Loss: 0.5442 | Test Acc: 0.6413
K=10 | Seed 30 | Epoch 014 | Loss: 0.5465 | Test Acc: 0.6457
K=10 | Seed 30 | Epoch 015 | Loss: 0.5327 | Test Acc: 0.6457
K=10 | Seed 30 | Epoch 016 | Loss: 0.5321 | Test Acc: 0.6457
K=10 | Seed 30 | Epoch 0

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▅▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▂▁
test_acc,▁▂▃▂▂▄▄▂▄▄▄▄▄▃▇▅▆▆▇▅▆▆▆▇▅▇▆▆▆▆▆▇▇▆▇█▇▆▃▆
train_acc,▁▄▄▅▅▆▇▆▇▆▆▇▇▇▇▆▇▇▇▆▇▇▇▇▆▇▆██▇▇██▇███▇▇▇
epoch,50
loss,0.49047
test_acc,0.66816
train_acc,0.7618
